# Get Retriver

In [1]:
import sys

# Add the directory to sys.path
sys.path.append('/home/qianyucazelles/internal_knowledge_chatbot/src/utils')
from doc_to_vertex_search import *

In [2]:
parameters = get_me_parameters('../vector_store_me_parameters/ecg_all_me.json')

In [3]:
embeddings = get_embeddings()

In [4]:
db = get_vector_store(parameters, embeddings)

In [5]:
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import (
    Namespace,
    NumericNamespace,
)
filters = [Namespace(name="department", allow_tokens=["dpo"]),
           Namespace(name="loader_method", allow_tokens=["unstructured"])
          ]


numeric_filters = [NumericNamespace(name="chunk_size", value_float=2000.0, op="EQUAL")]


In [6]:
retriever = db.as_retriever()

In [7]:
retriever.search_kwargs = {"filter": filters, "numeric_filter": numeric_filters}

In [8]:
q = "Je n'aurai pas le temps de traiter la demande de droit à l'oubli d'un utilisateur dans le mois règlementaire imposé par la CNIL. Ai-je un recours ?"

In [9]:
q2 = "J'ai des difficultés à comprendre les différentes obligations que j'ai vis-à-vis des sous-traitants qui travaillent pour moi. Pourrais-tu me faire un résumé ?"

In [33]:
retrieved_documents = retriever.invoke(q)

In [11]:
retrieved_documents = retriever.invoke(q2)

## chat model

In [22]:
def format_prompt(SYS_PROMPT, prompt,retrieved_documents):
  """using the retrieved documents we will prompt the model to generate our responses"""
  formatted_prompt = f"{SYS_PROMPT} \n Question:{prompt}\nContext: \n"
  separated_line = "-"*50+"\n"
  for idx,doc in enumerate(retrieved_documents) :
    formatted_prompt+= f"{separated_line} Ranked Document: {idx+1} \nName: {doc.metadata['document_name']}\nContent: {doc.page_content} \n"
  return formatted_prompt

In [20]:
SYS_PROMPT = """You are an assistant for answering questions.
You are given the extracted parts of several documents rank by relevance with document name and a question. Please pick the most relevant document related to the question then 
provide a conversational answer in the same language as the question. Always cite the referred document in your response.
If you don't know the answer, just say "I do not know." Don't make up an answer."""

In [35]:
formatted_prompt = format_prompt(SYS_PROMPT,q,retrieved_documents)

In [25]:
chat.send_message(formatted_prompt, **parameters)

En tant que responsable du traitement, vous devez veiller à ce que les sous-traitants que vous engagez présentent des garanties suffisantes quant à la mise en œuvre de mesures techniques et organisationnelles appropriées de manière à ce que le traitement réponde aux exigences du présent règlement et garantisse la protection des droits de la personne concernée.

Vous devez également conclure un contrat ou un autre acte juridique avec le sous-traitant qui définit l'objet et la durée du traitement, la nature et la finalité du traitement, le type de données à caractère personnel et les catégories de personnes concernées, et les obligations et les droits du responsable du traitement. Ce contrat ou cet autre acte juridique doit prévoir notamment que le sous-traitant ne traite les données à caractère personnel que sur instruction documentée du responsable du traitement, qu'il veille à ce que les personnes autorisées à traiter les données à caractère personnel s'engagent à respecter la confide

In [45]:
from langchain_google_vertexai import VertexAI
model = VertexAI(
    model_name="gemini-1.0-pro", #"gemini-1.0-pro", #"text-bison@002",
    max_output_tokens=1024,
    temperature=0.0,
    verbose=True,
    streaming = True
)

In [48]:
q

"Je n'aurai pas le temps de traiter la demande de droit à l'oubli d'un utilisateur dans le mois règlementaire imposé par la CNIL. Ai-je un recours ?"

In [47]:
from langchain_core.prompts import PromptTemplate

template = """{question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | model

print(chain.invoke({"question": formatted_prompt}))

## Oui, vous avez un recours si vous n'aurez pas le temps de traiter la demande de droit à l'oubli d'un utilisateur dans le mois règlementaire imposé par la CNIL.

**Voici les options qui s'offrent à vous :**

* **Demander une prolongation du délai.** Selon l'article 12 du RGPD, vous pouvez demander une prolongation de deux mois si la demande est complexe ou si vous avez reçu un nombre important de demandes. Vous devez informer la personne concernée de cette prolongation et des motifs du report dans un délai d'un mois à compter de la réception de la demande.
* **Déléguer la tâche à un tiers.** Vous pouvez déléguer la tâche de traitement de la demande de droit à l'oubli à un tiers, tel qu'un prestataire spécialisé. Assurez-vous que le tiers est soumis aux mêmes obligations que vous en matière de protection des données.
* **Informer la CNIL de l'impossibilité de respecter le délai.** Si vous ne pouvez pas respecter le délai, même en demandant une prolongation, vous devez en informer la C

# Test question refiner

In [49]:
refiner_template = "Given the following user query and conversation log, formulate a question that includes all the entities or subject mentioned would be the most relevant to provide the user with an answer from a knowledge base specifying the subject if it's mentioned.\n\nCONVERSATION LOG: \n{conversation}\n\nQuery: {query}\n\nRefined Query:"
refiner_prompt = PromptTemplate.from_template(refiner_template)

In [50]:
refiner_chain = refiner_prompt | model

In [51]:
conversation_string = ""
query = '''Je n'aurai pas le temps de traiter la demande de droit à l'oubli d'un utilisateur dans le mois règlementaire imposé par la CNIL. Ai-je un recours ?'''
response = '''1 month'''
conversation_string += "Human: "+query+ "\n"
conversation_string += "Bot: "+ response + "\n"

In [54]:
refiner_chain.invoke({"conversation": conversation_string,"query":"can you give more details"})

"## Refined Query:\n\n**Est-il possible de demander un délai supplémentaire à la CNIL pour traiter une demande de droit à l'oubli, si l'on ne peut pas la traiter dans le délai réglementaire d'un mois ?** \n\nThis refined query incorporates the following elements from the conversation log:\n\n* **Subject:** Droit à l'oubli (right to be forgotten)\n* **Entity:** CNIL (French data protection authority)\n* **Timeframe:** 1 month\n* **User's concern:** Inability to meet the deadline\n* **User's question:** Seeking a solution\n\nBy combining these elements, the refined query directly addresses the user's concern and seeks information about potential solutions within the context of the right to be forgotten and the CNIL's regulations. \n"